# 4주차 Multiclass SVM 구현 과제 - 16기 김주호

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

#IRIS 데이터 로드
iris =  sns.load_dataset('iris')
X= iris.iloc[:,:4] #학습할데이터
y = iris.iloc[:,-1] #타겟
print(y)

0         setosa
1         setosa
2         setosa
3         setosa
4         setosa
         ...    
145    virginica
146    virginica
147    virginica
148    virginica
149    virginica
Name: species, Length: 150, dtype: object


In [2]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=48)

In [3]:
def standardization(train, test):
    scaler = StandardScaler()
    train = scaler.fit_transform(train)
    test = scaler.transform(test)
    return train, test

X_train, X_test = standardization(X_train, X_test)

In [4]:
X_train

array([[ 0.78522493,  0.32015325,  0.77221097,  1.04726529],
       [-0.26563371, -1.29989934,  0.0982814 , -0.11996537],
       [ 0.43493872,  0.78302542,  0.94069336,  1.43634218],
       [-0.84944407,  0.78302542, -1.24957775, -1.28719604],
       [-0.38239578, -1.7627715 ,  0.15444219,  0.13941922],
       [ 0.55170079, -0.374155  ,  1.05301496,  0.7878807 ],
       [ 0.31817664, -0.14271892,  0.65988937,  0.7878807 ],
       [ 0.20141457, -0.374155  ,  0.43524618,  0.39880381],
       [-1.66677857, -0.14271892, -1.36189934, -1.28719604],
       [-0.14887164, -0.60559109,  0.21060299,  0.13941922],
       [-0.14887164, -1.06846325, -0.12636179, -0.24965767],
       [ 0.31817664, -0.60559109,  0.15444219,  0.13941922],
       [ 0.66846286, -0.83702717,  0.88453256,  0.91757299],
       [ 0.0846525 , -0.14271892,  0.77221097,  0.7878807 ],
       [-0.49915786, -0.14271892,  0.43524618,  0.39880381],
       [-0.26563371, -0.60559109,  0.65988937,  1.04726529],
       [ 2.18636979,  1.

In [5]:
X_test

array([[-0.14887164, -0.374155  ,  0.26676379,  0.13941922],
       [ 0.31817664, -0.60559109,  0.54756778,  0.00972692],
       [ 0.31817664, -1.06846325,  1.05301496,  0.26911151],
       [-1.5500165 , -1.7627715 , -1.36189934, -1.15750374],
       [ 0.0846525 ,  0.32015325,  0.60372857,  0.7878807 ],
       [ 0.78522493, -0.14271892,  0.99685416,  0.7878807 ],
       [-0.84944407,  1.70876975, -1.24957775, -1.15750374],
       [ 0.20141457, -0.14271892,  0.60372857,  0.7878807 ],
       [-0.38239578,  2.63451409, -1.30573855, -1.28719604],
       [-0.38239578, -1.29989934,  0.15444219,  0.13941922],
       [ 0.66846286,  0.08871717,  0.99685416,  0.7878807 ],
       [-0.38239578,  1.0144615 , -1.36189934, -1.28719604],
       [-0.49915786,  0.78302542, -1.13725615, -1.28719604],
       [ 0.43493872, -0.60559109,  0.60372857,  0.7878807 ],
       [ 0.55170079, -1.7627715 ,  0.37908538,  0.13941922],
       [ 0.55170079,  0.55158933,  0.54756778,  0.52849611],
       [-1.19973028,  0.

In [6]:
# 결측치 없음
iris.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [7]:
y_train.value_counts()

setosa        41
virginica     40
versicolor    39
Name: species, dtype: int64

## SVM으로 훈련하기
###  one vs rest 방식으로 클래스 예측

**< 구현 과정 >**
* **1. 클래스가 n가지일 때, 이를 원핫인코딩하여 n개의 타겟값 열을 만든다**
* **2. n개의 binary SVM 학습기를 만들어 각각을 훈련한 후 각 학습기의 예측값의 배열을 변수에 저장해둔다** 
* **3. 예측값 배열에서 하나의 열 안에 1이 하나인 열은 1에 해당하는 원 데이터의 타겟값을 result 리스트 안에 append한다** 
* **4. 예측값 배열에서 하나의 열 안에 1이 두개 이상이거나 모두 0인 열은 decision_function()을 통해 가장 큰 값의 타겟값을 result에 append한다**
* **5. 위 3,4번의 결과를 조합하여 multiclass SVM 구현한다** 

In [8]:
class SVM_ovr:
    def __init__(self,num_classes,kernel,C,gamma):
        self.num_classes=num_classes
        self.svm=[SVC(kernel=kernel,C=C,gamma=gamma) for i in range(num_classes)]
        
    def fit_predict(self, X_train, y_train, X_test):
        # y_train 원핫인코딩
        y_train=pd.get_dummies(y_train)
        # 학습기의 예측값을 넣을 빈 배열 pred
        pred=np.zeros((self.num_classes,len(X_test)))
        
        # (A vs Rest) svm모델, (B vs Rest) svm모델, (C vs Rest) svm모델, ... 등을 학습시키고 각각에 대한 예측값을 pred라는 배열에 저장
        for i in range(len(self.svm)):
            self.svm[i].fit(X_train,y_train.iloc[:,i])
            pred[i]=self.svm[i].predict(X_test)
            
        result=[]
        for i in range(len(X_test)):
            # 예측값 배열에서 각 열의 합이 1인 경우(ex. [1,0,0] [0,1,0] [0,0,1])
            if sum(pred[:,i])==1:
                result.append(np.argmax(pred[:,i],axis=0))
            # 각 열의 합이 1이 아닌 경우(ex. [1,1,0],[0,0,0],[1,1,1]) => 흔한 상황은 아닐 것으로 예상됨
            else:
                # **동점의 상황에서만 거리계산을 하도록 함**
                decision_score=np.zeros((self.num_classes,1))
                for ind in range(self.num_classes):
                    decision_score[ind]=self.svm[ind].decision_function(X_test[i].reshape(1,-1))
                result.append(np.argmax(decision_score))
        print(result)
        return result

**위 클래스는 fit() 함수와 predict() 함수를 합쳐 fit_predict() 함수를 만들었으며 큰 차이는 아니지만 데이터프레임이나 리스트로의 변환을 최소화하고 주로 배열의 형태로 통일하여 메모리 사용량을 줄이고자 했다. 또한, 데이터의 크기가 커질 경우, decision_function()을 통해 결정경계를 구하는 것이 수행시간을 늘릴 수 있다고 생각하여 동점이 발생하는 상황에서만 decision_function을 수행하도록 코드를 작성했다.**

**SVM모델에서 decision_function은 data point와 hyperplane 사이의 거리와 비례하는 값을 갖는다.**

If decision_function_shape=’ovo’, the function values are proportional to the distance of the samples X to the separating hyperplane. If the exact distances are required, divide the function values by the norm of the weight vector (coef_). See also this question for further details. If decision_function_shape=’ovr’, the decision function is a monotonic transformation of ovo decision function.  
출처: https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html

### 구현한 multiclass SVM 정확도 구하기

In [9]:
# y_test 타겟값 열 원핫인코딩 (One vs Rest에 맞도록 일반적인 원핫인코딩 진행)
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
y_test_le=encoder.fit_transform(y_test)
print(y_test_le)

[1 1 2 0 1 2 0 2 0 1 2 0 0 2 1 1 0 1 1 2 0 2 1 1 2 0 0 2 2 1]


In [10]:
clf=SVM_ovr(num_classes=3,kernel='rbf',C=5,gamma=5)
y_pred=clf.fit_predict(X_train,y_train,X_test)
accuracy_score(y_test_le,y_pred)

[1, 1, 1, 2, 2, 2, 0, 2, 0, 1, 2, 0, 0, 2, 1, 1, 0, 1, 2, 2, 0, 2, 1, 1, 2, 0, 0, 2, 2, 1]


0.8666666666666667

### 수행시간 비교를 위한 측정

In [345]:
rt_list_me=[]

In [355]:
import time
start = time.time()

clf=SVM_ovr(num_classes=3,kernel='rbf',C=5,gamma=5)
clf.fit_predict(X_train,y_train,X_test)

run_time_me=time.time() - start
print("time :", run_time_me)

rt_list_me.append(run_time_me)

[1, 1, 1, 2, 2, 2, 0, 2, 0, 1, 2, 0, 0, 2, 1, 1, 0, 1, 2, 2, 0, 2, 1, 1, 2, 0, 0, 2, 2, 1]
time : 0.008974313735961914


In [356]:
# 10번 수행하여 각 수행시간을 리스트에 더함
rt_list_me

[0.008976221084594727,
 0.011969327926635742,
 0.010970592498779297,
 0.01096963882446289,
 0.009974241256713867,
 0.00997161865234375,
 0.015956878662109375,
 0.01196599006652832,
 0.00897526741027832,
 0.008974313735961914]

In [385]:
# 수행시간 10번을 평균함
print(f'평균적으로 {round(np.mean(rt_list_me),3)}초 걸린다')

평균적으로 0.011초 걸린다


**15기 우수코드의 클래스와 수행시간 비교**

In [76]:
class SVM_OVR:
    def __init__(self, num_classes, kernel, C, gamma):
        self.num_classes = num_classes
        self.clfs = [SVC(kernel = kernel, C = C, gamma = gamma) for _ in range(num_classes)]
        self.classes = None
        
    def fit(self, X_train, y_train):
        y_train = pd.get_dummies(y_train)
        for i in range(self.num_classes):
            self.clfs[i].fit(X_train,y_train.iloc[:,i]) 
            # 각 클래스별로 인지 아닌지를 판단하는 분류기를 학습시킵니다.
        self.classes = y_train.columns
    
    def predict(self, X_test):
        pred_df = pd.DataFrame([svm.predict(X_test) for svm in self.clfs]).T # 각 클래스 별 예측값
        decisions = np.array([svm.decision_function(X_test) for svm in self.clfs]).T # 각 클래스 별 거리
        
        final_pred = []
        for i in range(len(pred_df)):
            # 예측 중 하나의 클래스만 맞다고 판단한 경우
            # 맞다고 판단된 클래스를 final_pred 리스트에 넣어준다
            if sum(pred_df.iloc[i]) == 1:
                label = pred_df.iloc[i][pred_df.iloc[i] == 1].index[0]
                final_pred.append(self.classes[label])
            else:
                label = np.argmax(decisions[i])
                final_pred.append(self.classes[label])
        
        print(final_pred)
        return final_pred

In [358]:
rt_list_15=[]

In [368]:
import time
start = time.time()

clf = SVM_OVR(num_classes = 3, kernel = 'rbf', C = 5, gamma = 5)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)

run_time_15=time.time() - start
print("time :", run_time_15)

rt_list_15.append(run_time_15)

['versicolor', 'versicolor', 'versicolor', 'virginica', 'virginica', 'virginica', 'setosa', 'virginica', 'setosa', 'versicolor', 'virginica', 'setosa', 'setosa', 'virginica', 'versicolor', 'versicolor', 'setosa', 'versicolor', 'virginica', 'virginica', 'setosa', 'virginica', 'versicolor', 'versicolor', 'virginica', 'setosa', 'setosa', 'virginica', 'virginica', 'versicolor']
time : 0.031958580017089844


In [369]:
# 10번 수행하여 각 수행시간을 리스트에 더함
rt_list_15

[0.04091691970825195,
 0.02892327308654785,
 0.02994990348815918,
 0.028920650482177734,
 0.04092860221862793,
 0.029917240142822266,
 0.028920650482177734,
 0.029915332794189453,
 0.03290891647338867,
 0.031958580017089844]

In [386]:
print(f'평균적으로 {round(np.mean(rt_list_15),3)}초 걸린다')

평균적으로 0.032초 걸린다


최대한 다른 변인은 통제하려고 했으나 모든 변인을 통제한 것은 아니기에 정확하지 않을 수 있지만 평균적으로 약 3배 정도 수행시간을 단축했다고 볼 수 있다. 다른 데이터셋에서도 코드의 효율성이 좋은지 확인해봐야 할 것으로 보인다.

### 다른 데이터셋(mnist 데이터셋)을 통해 수행시간이 실제로 단축되었는지 확인

**SVM_OvR 클래스 (동점이 발생하는 상황에서만 거리 계산을 한 경우, 제가 만든 클래스)**

In [102]:
from sklearn.datasets import load_digits
mnist = load_digits()
X = mnist.data
y = mnist.target

In [38]:
X/=16

In [30]:
np.unique(y)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [29]:
X_train.shape

(1437, 64)

In [47]:
# 나의 모델의 수행시간 담아놓을 리스트
rt_list_1=[]

In [69]:
import time
start = time.time()

clf = SVM_ovr(num_classes = 10, kernel = 'rbf', C = 1, gamma = 1)
pred = clf.fit_predict(X_train, y_train, X_test)

run_time_1=time.time() - start
print("time :", run_time_1)

rt_list_1.append(run_time_1)

[6, 0, 7, 2, 6, 5, 9, 9, 3, 7, 1, 6, 1, 9, 6, 6, 3, 1, 7, 7, 6, 4, 1, 9, 7, 7, 2, 3, 4, 2, 6, 3, 4, 1, 5, 8, 2, 5, 0, 9, 2, 4, 5, 9, 9, 8, 4, 2, 1, 9, 5, 6, 6, 8, 2, 8, 4, 3, 0, 5, 5, 9, 1, 9, 4, 6, 6, 0, 9, 3, 3, 4, 3, 5, 0, 5, 5, 9, 9, 9, 0, 9, 9, 7, 3, 3, 1, 0, 4, 9, 8, 1, 7, 6, 3, 2, 9, 7, 6, 5, 1, 6, 8, 7, 4, 7, 2, 6, 3, 1, 6, 7, 1, 7, 8, 2, 4, 3, 0, 5, 9, 6, 0, 5, 2, 7, 0, 0, 1, 2, 7, 5, 8, 5, 1, 7, 1, 0, 7, 4, 5, 5, 8, 9, 5, 5, 4, 6, 8, 5, 6, 2, 9, 1, 4, 9, 4, 6, 6, 4, 9, 7, 6, 3, 3, 6, 7, 1, 9, 0, 1, 7, 3, 1, 5, 0, 2, 5, 3, 5, 9, 7, 8, 4, 0, 2, 4, 7, 3, 8, 9, 0, 5, 5, 1, 1, 3, 8, 0, 5, 5, 2, 6, 0, 8, 9, 1, 6, 4, 3, 8, 4, 2, 6, 2, 8, 1, 0, 3, 0, 0, 7, 0, 2, 8, 0, 2, 8, 9, 2, 6, 5, 9, 1, 4, 9, 0, 3, 1, 3, 0, 6, 3, 9, 9, 0, 8, 0, 8, 7, 9, 6, 2, 8, 0, 1, 6, 1, 3, 5, 2, 4, 7, 1, 6, 4, 1, 1, 7, 1, 6, 6, 8, 3, 7, 9, 8, 1, 7, 1, 1, 1, 7, 8, 2, 7, 3, 9, 4, 8, 9, 3, 3, 5, 2, 8, 3, 2, 9, 1, 4, 3, 5, 0, 7, 9, 0, 5, 5, 7, 7, 0, 2, 1, 1, 1, 0, 7, 7, 7, 5, 0, 3, 9, 7, 0, 1, 6, 2, 2, 9, 1, 0, 

In [67]:
# 정확도 출력
accuracy_score(y_test, pred)

0.9888888888888889

In [70]:
rt_list_1

[0.2433462142944336,
 0.23935818672180176,
 0.23739361763000488,
 0.26525020599365234,
 0.23436903953552246,
 0.2353672981262207,
 0.25631165504455566,
 0.25930190086364746,
 0.23436737060546875,
 0.23038315773010254]

In [71]:
print(f'평균적으로 {round(np.mean(rt_list_1),3)}초 걸린다')

평균적으로 0.244초 걸린다


**SVM_OVR 클래스 (모든 행 데이터에 대해 거리 계산을 한 경우, 15기 우수코드 클래스)**

In [105]:
mnist = load_digits()
X = pd.DataFrame(mnist.data)
y = pd.Series(mnist.target)
X /= 16
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=317)

In [74]:
# 15기 우수코드 모델을 담을 리스트
rt_list_2=[]

In [95]:
import time
start = time.time()

clf = SVM_OVR(num_classes = 10, kernel = 'rbf', C = 1, gamma = 1)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)

run_time_2=time.time() - start
print("time :", run_time_2)

rt_list_2.append(run_time_2)

[6, 0, 7, 2, 6, 5, 9, 9, 3, 7, 1, 6, 1, 9, 6, 6, 3, 1, 7, 7, 6, 4, 1, 9, 7, 7, 2, 3, 4, 2, 6, 3, 4, 1, 5, 8, 2, 5, 0, 9, 2, 4, 5, 9, 9, 8, 4, 2, 1, 9, 5, 6, 6, 8, 2, 8, 4, 3, 0, 5, 5, 9, 1, 9, 4, 6, 6, 0, 9, 3, 3, 4, 3, 5, 0, 5, 5, 9, 9, 9, 0, 9, 9, 7, 3, 3, 1, 0, 4, 9, 8, 1, 7, 6, 3, 2, 9, 7, 6, 9, 1, 6, 8, 7, 4, 7, 2, 6, 3, 1, 6, 7, 1, 7, 8, 2, 4, 3, 0, 5, 9, 6, 0, 5, 2, 7, 0, 0, 1, 2, 7, 5, 8, 5, 1, 7, 1, 0, 7, 4, 5, 5, 8, 9, 5, 5, 4, 6, 8, 5, 6, 2, 9, 1, 4, 9, 4, 6, 6, 4, 9, 7, 6, 3, 3, 6, 7, 1, 9, 0, 1, 7, 3, 1, 5, 0, 2, 5, 3, 5, 9, 7, 8, 4, 0, 2, 4, 7, 3, 8, 9, 0, 5, 5, 1, 1, 3, 8, 0, 5, 5, 2, 6, 0, 8, 9, 1, 6, 4, 3, 8, 4, 2, 6, 2, 8, 1, 0, 3, 0, 0, 7, 0, 2, 8, 0, 2, 8, 9, 2, 6, 5, 5, 1, 4, 9, 0, 3, 1, 3, 0, 6, 3, 9, 9, 0, 8, 0, 8, 7, 9, 6, 2, 8, 0, 1, 6, 1, 3, 5, 2, 4, 7, 1, 6, 4, 1, 1, 7, 1, 6, 6, 8, 3, 7, 9, 8, 1, 7, 1, 1, 1, 7, 8, 2, 7, 3, 9, 4, 8, 9, 3, 3, 5, 2, 8, 3, 2, 9, 1, 4, 3, 5, 0, 7, 9, 0, 5, 5, 7, 7, 0, 2, 1, 1, 1, 0, 7, 7, 7, 5, 0, 3, 9, 7, 0, 1, 6, 2, 2, 9, 1, 0, 

In [96]:
rt_list_2

[2.2310431003570557,
 2.2290101051330566,
 2.1791865825653076,
 2.1641883850097656,
 2.1572067737579346,
 2.175159215927124,
 2.232038974761963,
 2.1841299533843994,
 2.2031173706054688,
 2.139252185821533]

In [97]:
print(f'평균적으로 {round(np.mean(rt_list_2),3)}초 걸린다')

평균적으로 2.189초 걸린다


## 결론

* **결론적으로 iris 데이터보다 크기가 큰 mnist 데이터에 대해서 SVM_OVR 클래스가 SVM_ovr 클래스보다 수행시간이 9배 정도 더 걸리는 것으로 보아, 필요한 경우에만 decision_function()을 사용함으로써 계산 시간을 많이 절약할 수 있음을 알 수 있다.**  


* **이번 과제에서는 One vs Rest방식으로만 Multiclass SVM을 구현했다. One vs One 방식과의 비교나 하이퍼파라미터의 조정을 통해 정확도 개선에 집중하기보다는 코드를 효율화하여 시간 복잡도와 메모리 사용량을 줄이는 쪽으로 집중하여 과제를 수행해보았다.**


* **앞으로 파이썬 알고리즘을 더 공부해서 위에서 짠 SVM_ovr 클래스 코드의 시간 복잡도와 사용하는 메모리양을 더욱 줄여보고싶다.**